<br>
<img src="Images/tvz_logo.png">
<br>

<center><h1>noSQL, BigData analiza #Stick2Me proizvoda</h1></center>
<center><h2>Dominik Košćica</h2></center>
<br>
<br>
<br>
<center><h2>Često gubite ili vam kradu ključeve, torbe, novčanike, kišobrane, kape…? Želite pratiti vašu djecu ili kućne ljubimce?</h2></center>
<br>
<img src="Images/stick2me_logo.png">
<center><h1>#Stick2Me</h1></center>
## Ključne značajke
<ul>
  <li>Aplikacija + naljepnica čine #Stick2Me</li>
  <li>Naljepnica koristi bluetooth beacon tehnologiju, a aplikacija će biti dostupna za iOS i Android uređaje</li>
  <li>Aplikacija prijavljuje korisniku udaljenost od svog novčanika, ključeva, kišobrana, torbe, djeteta ili kućnog ljubimca</li>
  <li>Ovisno o postavkama aplikacija obaviještava/upozorava korisnika o prekoračenju zadane udaljenosti od vaših predmeta</li>
</ul>

## Korištene tehnologije, jezici, servisi, alati i biblioteke

<ul>
  <li><a href="http://www.tweepy.org/">Tweepy</a><br>Python library za pristup Twitter API-u.</li>
  <br>
  <li><a href="https://api.mongodb.com/python/current/">PyMongo</a></li>
  <br>
  <li><a href="https://www.python.org/">Python 3.6</a></li>
  <br>
  <li><a href="https://www.mongodb.com/">MongoDB</a></li>
  <br>
  <li><a href="https://robomongo.org/">RoboMongo</a></li>
</ul>


## Generalna struktura projekta

<ul> 
  <li>
      <h4>Util.py</h4>
      <p>Sadrži kod za formatirani ispis u konzolu.</p>
  </li>
  <li>
      <h4>TweepyWrapper.py</h4>
      <p>Sadrži kod za autentifikaciju na tweeter API, metode za dohvaćanje informacija o korisniku, njegovim followerima, tweetovima kao i pomoćne metode za ispis relevantnih informacija vezani za twitter API.</p>
  </li>
  <li>
      <h4>PyMongoWrapper.py</h4>
      <p>Sadrži kod za spajanje na MongoDB bazu, transformiranje Tweeta objecta dobiven sa Tweepya u prihvatljiv MongoDB document oblik, unos u bazu kao i ispis iz baze.</p>
  </li>
    <li>
      <h4>Main.py</h4>
      <p>Main klasa u kojoj se instanciraju TweepyWrapper, PyMongoWrapper, NLTKWrapper klase i pozivaju sve relevanten metode za analizu #Stick2Me proizvoda.</p>
  </li> 
</ul>

### Util.py

In [ ]:
"""
Print methods
"""
def show_header(text):
    print("\n")
    print("###############################################")
    print(text)
    print("###############################################")
    
def print_tweet(tweet):
    print("Date:", tweet.created_at)
    print("Name:", tweet.user.name)
    print("Screen name:", tweet.user.screen_name)
    print("Text:", tweet.text)
    print()
    
def print_user(user):
    print("Id:", user.id)
    print("Created at:", user.created_at)
    print("Name:", user.name)
    print("Screen name:", user.screen_name)
    print("Description:", user.description)
    print("Followers count:", user.followers_count)
    print("Friends count:", user.friends_count)
    print("Favorites count:", user.favourites_count)
    print("Statuses count:", user.statuses_count)
    print()


### TweepyWrapper.py

In [6]:
import tweepy

class TweepyWrapper:
    
    #Constants
    CONSUMER_KEY = "GcCRUCVDeJf0H6zG7k3BqCsXV"
    CONSUMER_SECRET = "I6RvI3tFjnd5XL1BgfDNqqJ1bVSXinBYooTOGJpg7KCug9mlLt"
    ACCESS_TOKEN = "810627882104057856-nxELiESEiHYvZCbymNDI98us80A2Ayo"
    ACCESS_TOKEN_SECRET = "1uAWrovjiDQvztHfiG9K6C94MrIMs3GH1dncuJO5DXiAW"
    
    #Members
    __followers = []
    __mostInfluentialFollowers = []

    __allMyTweets = []
    __allTweetsFromMyFollowers = []
        
    def __init__(self):
        #OAuth
        auth = tweepy.OAuthHandler(self.CONSUMER_KEY, self.CONSUMER_SECRET)
        auth.set_access_token(self.ACCESS_TOKEN,  self.ACCESS_TOKEN_SECRET)
        
        #Construct the API instance
        self.api = tweepy.API(auth)
        
    """
    Print methods
    """
    def print_user_information(self):
        show_header("Get all the information about @TVZ_dkoscica")
        user = self.api.me()
        print_user(user)
        
    """
    Follower methods
    """
    def get_my_followers(self):
        if not self.__followers:
            self.__followers = self.api.followers()
            return self.__followers
        return self.__followers
    
    def get_most_influential_followers(self):
        if not self.__mostInfluentialFollowers:
            self.__mostInfluentialFollowers = sorted(self.get_my_followers(), 
                                                     key=lambda user: user.statuses_count, reverse = True)
            return self.__mostInfluentialFollowers
        return self.__mostInfluentialFollowers
        
    def print_all_my_followers(self):
        show_header("All my followers")
        for follower in self.get_my_followers():
            print_user(follower)
            
    def print_most_influential_followers(self):
        show_header("Top 10 most influential followers")
        for follower in self.get_most_influential_followers()[:10]:
            print_user(follower)  
     
    """
    Tweets methods
    """
    def get_all_my_tweets(self):
        if not self.__allMyTweets:
            self.__allMyTweets = tweepy.Cursor(self.api.user_timeline).items()
            return self.__allMyTweets
        return self.__allMyTweets
        
    def get_all_tweets_from_user(self, screen_name):
        return self.api.user_timeline(screen_name = screen_name, count=500) 
        
    def get_all_tweets_from_my_followers(self):
        if not self.__allTweetsFromMyFollowers:
            #allTweetsFromMyFollowers.append(allMyTweets)
            for follower in self.get_my_followers():
                allTweetsFromUser = self.get_all_tweets_from_user(follower.screen_name)
                self.__allTweetsFromMyFollowers.extend(allTweetsFromUser)
            return self.__allTweetsFromMyFollowers 
            
        return self.__allTweetsFromMyFollowers 
            
    def get_all_tweets_which_contain_Stick2Me(self):
        tweetsThatContainStick2Me = []
        for tweet in self.get_all_tweets_from_my_followers():
            if "Stick2Me" in tweet.text:
                tweetsThatContainStick2Me.append(tweet)
        return tweetsThatContainStick2Me

    def print_all_tweets_from_me(self):
        show_header("All tweets from @TVZ_dkoscica")
        for tweet in self.get_all_my_tweets():
            print_tweet(tweet)
            
    def print_retweets_of_me(self):
        retweets = self.api.retweets_of_me()
        numberOfRetweets = str(len(retweets))
        show_header("Retweets of me\nNumber of retweets: "  + numberOfRetweets)
        for tweet in retweets:
            print_tweet(tweet)
                           
    def print_all_tweets_which_contain_Stick2Me(self):
        #+ str(len(self.get_all_tweets_which_contain_Stick2Me_or_TVZ_dkoscica))
        show_header("Tweets that contain Stick2Me or @TVZ_dkoscica\nNumber of tweets: ")
        for tweet in self.get_all_tweets_which_contain_Stick2Me():
            print_tweet(tweet)

### MongoDbWrapper.py

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 31 20:25:37 2017

@author: Dominik
"""

import pymongo
from pymongo import MongoClient
import pprint
import datetime

class PyMongoWrapper:
    
    #Never do this in production, 
    #the DB username an password must not be visible in code!!!
    DB_USERNAME = "admin"
    DB_PASSWORD = "stick2Me.Admin"
    
    HOST = "ds149069.mlab.com"
    PORT = str(49069)
    DATABASE_NAME = "stick2me_database"
    COLLECTION_NAME = "tweets.all"
    
    def __init__(self):
        
        connectionUri = "mongodb://" + self.DB_USERNAME + ":" + self.DB_PASSWORD + "@" + self.HOST + ":" + self.PORT + "/" + self.DATABASE_NAME
        
        self.client = MongoClient(connectionUri)
        self.database = self.client[self.DATABASE_NAME]
        self.collection = self.database[self.COLLECTION_NAME]
        self.print_collection(self.COLLECTION_NAME)
        print("Database connected!")
            
    """
    Insert all tweets into one MongoDB collection, 
    this collections will be later split by weeks via direct MongoDB commands
    """
    def insert_all_tweets(self, tweets):
        tweetDocuments = self.transform_tweets_to_tweet_documents(tweets)
        #tweetDocumentsSet = set(tweetDocuments) #Use set to remove possible duplicates
        self.collection.insert_many(tweetDocuments)
        print("insertAllTweets")
    
    def transform_tweets_to_tweet_documents(self, tweets):
        tweetDocuments = []
        for tweet in tweets:
            tweetDocuments.append(self.create_tweet_document(tweet))
        return tweetDocuments
    
    def create_tweet_document(self, tweet):
        return {"created_at": tweet.created_at,
                "name": tweet.user.name,
                "screen_name:": tweet.user.screen_name,
                "text:": tweet.text
                }
    
    """
    Print methods
    """
    def print_collection(self, collection_name):
        show_header("Collections")
        pprint.pprint(self.collection)

    def print_one_tweet(self):
        show_header("Find one Tweet")
        user = self.collection.find_one()
        pprint.pprint(user)

    def print_all_collection_tweets(self):
        show_header("All collection Tweets")
        for item in self.collection.find():
            pprint.pprint(item)

### Main.py

In [16]:
def main():

    tweepyWrapper = TweepyWrapper()
    
    tweepyWrapper.print_user_information()
    
    tweepyWrapper.print_all_my_followers()
    tweepyWrapper.print_most_influential_followers()
    
    #tweepyWrapper.print_all_tweets_from_me()
    #tweepyWrapper.print_retweets_of_me()

    allTweetsWhichContainStick2MeOrTVZ_dkoscica = tweepyWrapper.get_all_tweets_which_contain_Stick2Me()
    show_header("Number of Tweets that contain Stick2Me: "  + str(len(allTweetsWhichContainStick2MeOrTVZ_dkoscica)))
    tweepyWrapper.print_all_tweets_which_contain_Stick2Me()

    pyMongoWrapper = PyMongoWrapper()
    pyMongoWrapper.insert_all_tweets(allTweetsWhichContainStick2MeOrTVZ_dkoscica)    
    pyMongoWrapper.print_one_tweet()
    #pyMongoWrapper.print_all_collection_tweets()
    
if __name__ == "__main__": main()



###############################################
Get all the information about @TVZ_dkoscica
###############################################
Id: 810627882104057856
Created at: 2016-12-18 23:28:42
Name: #Stick2Me
Screen name: TVZ_dkoscica
Description: Često gubite ili vam kradu ključeve, torbe, novčanike, kišobrane? Želite pratiti vašu djecu ili kućne ljubimce?
#Stick2Me je rješenje za vaše probleme :)
Followers count: 33
Friends count: 34
Favorites count: 16
Statuses count: 174



###############################################
All my followers
###############################################
Id: 818226336238080000
Created at: 2017-01-08 22:42:15
Name: The Real Hoverboard
Screen name: TVZ_fvinkovic
Description: 
Followers count: 17
Friends count: 36
Favorites count: 5
Statuses count: 31

Id: 810652583597637633
Created at: 2016-12-19 01:06:51
Name: TVZ NoSQL Potočki
Screen name: tvz_cuvardoma
Description: 
Followers count: 33
Friends count: 30
Favorites count: 0
Statuses count: 1

Id: 8

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it